### Partie 1 : Ingestion et nettoyage avec Spark (4-5h)

**Competence evaluee : C2.1 - Collecter des donnees en respectant les normes et standards**

#### Etape 1.1 : Exploration initiale
- Charger les donnees de consommation avec PySpark

In [33]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import TimestampType
from pathlib import Path
from datetime import datetime

# Chemins des donnees
DATA_DIR = Path("../data_ecf").resolve()
CONSOMMATIONS_PATH = DATA_DIR / "consommations_raw.csv"

In [20]:
builder: SparkSession.Builder = SparkSession.builder
spark = (
    builder
    .appName("01_exploration_spark")
    .master("local[*]")
    .config("spark.driver.memory", "2g")
    .config("spark.sql.shuffle.partitions", "8")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("WARN")

print(f"Spark version: {spark.version}")
print(f"Spark UI: {spark.sparkContext.uiWebUrl}")

Spark version: 3.5.7
Spark UI: http://host.docker.internal:4040


In [21]:
df_consommations_raw = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv(CONSOMMATIONS_PATH.as_posix())
)

print(f"Nombre de lignes: {df_consommations_raw.count()}")
print(f"Nombre de colonnes: {len(df_consommations_raw.columns)}")

Nombre de lignes: 7758868
Nombre de colonnes: 5


- Analyser le schema infere et identifier les problemes de typage

In [22]:
print("Schema infere :")
df_consommations_raw.printSchema()

Schema infere :
root
 |-- batiment_id: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- type_energie: string (nullable = true)
 |-- consommation: string (nullable = true)
 |-- unite: string (nullable = true)



In [23]:
print("10 premières lignes du DataFrame :")
df_consommations_raw.show(10, truncate=False)

10 premières lignes du DataFrame :
+-----------+-------------------+------------+------------+-----+
|batiment_id|timestamp          |type_energie|consommation|unite|
+-----------+-------------------+------------+------------+-----+
|BAT0141    |2023-12-21 13:00:00|gaz         |342.34      |kWh  |
|BAT0080    |08/08/2023 13:00   |gaz         |1256.73     |kWh  |
|BAT0122    |06/13/2024 11:00:00|eau         |133.57      |m3   |
|BAT0033    |2023-06-25 00:00:00|eau         |0.23        |m3   |
|BAT0064    |11/29/2024 04:00:00|gaz         |12.26       |kWh  |
|BAT0052    |09/15/2024 18:00:00|gaz         |701.01      |kWh  |
|BAT0097    |2024-01-25 12:00:00|gaz         |444.87      |kWh  |
|BAT0073    |27/03/2023 13:00   |electricite |123.97      |kWh  |
|BAT0084    |03/12/2024 17:00:00|eau         |17.88       |m3   |
|BAT0022    |2024-10-30 03:00:00|eau         |0.80        |m3   |
+-----------+-------------------+------------+------------+-----+
only showing top 10 rows



La visualisation des premières lignes du DataFrame nous permet d'identifier que la colonne timestamps contient plusieurs formats de date et que la colonne consommation devrait être casté en float.

In [24]:
df_consommation_non_numeric = df_consommations_raw.filter(
    ~F.col("consommation").rlike("^-?[0-9]+[.,]?[0-9]*$")
)

print(f"Nombre de valeurs non numeriques dans la colonne consommation : {df_consommation_non_numeric.count()}")
df_consommation_non_numeric.select("consommation").distinct().show()

df_consommation_numeric_with_comma = df_consommations_raw.filter(F.col("consommation").rlike("^-?[0-9]+[.,]?[0-9]*$")).filter(F.col("consommation").contains(","))

print(f"Nombre de valeurs avec virgule dans la colonne consommation : {df_consommation_numeric_with_comma.count():,}")

Nombre de valeurs non numeriques dans la colonne consommation : 38975
+------------+
|consommation|
+------------+
|        null|
|         N/A|
|      erreur|
|         ---|
+------------+

Nombre de valeurs avec virgule dans la colonne consommation : 925,392


La colonne consommation est en string car elle contient des nombres avec séparateur virgule et des valeurs textuelles. Les valeurs textuées de la colonne consommation sont "null", "N/A", "erreur" et "---".

In [ ]:
df_consommation_numeric = df_consommations_raw.withColumn(
    "consommation_clean",
    F.regexp_replace(F.col("consommation"), ",", ".").cast("double")
)

def parse_multi_format_timestamp(timestamp_str: str) -> datetime|None:
    if timestamp_str is None:
        return None
    formats = [
        "%Y-%m-%d %H:%M:%S",
        "%d/%m/%Y %H:%M",
        "%m/%d/%Y %H:%M:%S",
        "%Y-%m-%dT%H:%M:%S",
    ]
    for fmt in formats:
        try:
            return datetime.strptime(timestamp_str, fmt)
        except ValueError:
            continue
    return None

parse_timestamp_udf = F.udf(parse_multi_format_timestamp, TimestampType())

df_consommation_numeric_timestamp = df_consommation_numeric.withColumn(
    "timestamp_clean",
    parse_timestamp_udf(F.col("timestamp"))
)


In [36]:
null_counts = df_consommation_numeric_timestamp.select([
    F.count(F.when(F.col(c).isNull() | (F.trim(F.col(c)) == "") | (F.trim(F.lower(F.col(c))).isin("null", "n/a", "erreur", "---")), c)).alias(c)
    for c in df_consommation_numeric_timestamp.columns
])

print("Nombre de valeurs nulles/vides par colonne:")
null_counts.show()

Nombre de valeurs nulles/vides par colonne:
+-----------+---------+------------+------------+-----+------------------+---------------+
|batiment_id|timestamp|type_energie|consommation|unite|consommation_clean|timestamp_clean|
+-----------+---------+------------+------------+-----+------------------+---------------+
|          0|        0|           0|       38975|    0|             38975|              0|
+-----------+---------+------------+------------+-----+------------------+---------------+



- Calculer les statistiques descriptives par type d'energie

In [27]:
stats_by_energy_type = df_consommation_numeric.filter(F.col("consommation_clean").isNotNull()) \
    .groupBy("type_energie") \
    .agg(
        F.count("*").alias("count"),
        F.round(F.mean("consommation_clean"), 2).alias("mean"),
        F.round(F.stddev("consommation_clean"), 2).alias("stddev"),
        F.round(F.min("consommation_clean"), 2).alias("min"),
        F.round(F.max("consommation_clean"), 2).alias("max"),
        F.round(F.expr("percentile(consommation_clean, 0.25)"), 2).alias("q1"),
        F.round(F.expr("percentile(consommation_clean, 0.5)"), 2).alias("q2"),
        F.round(F.expr("percentile(consommation_clean, 0.75)"), 2).alias("q3"),
    ) \
    .orderBy("type_energie")

print("Statistiques par type d'énergie:")
stats_by_energy_type.show()

Statistiques par type d'énergie:
+------------+-------+------+-------+--------+--------+-----+------+------+
|type_energie|  count|  mean| stddev|     min|     max|   q1|    q2|    q3|
+------------+-------+------+-------+--------+--------+-----+------+------+
|         eau|2573156|204.36|2398.57| -657.01|49999.23| 1.72|  7.52| 22.87|
| electricite|2573364|430.64|2429.51|-4003.35|49999.13|29.33|108.56|306.35|
|         gaz|2573373|560.94|2465.81|-5963.49|49999.49|43.63|160.57|454.66|
+------------+-------+------+-------+--------+--------+-----+------+------+



In [28]:
print("Valeurs negatives dans la colonne consommation:")
df_consommation_numeric.filter(F.col("consommation_clean") < 0).groupBy("type_energie").count().show()

print("Valeurs abérantes > 15 000 dans la colonne consommation :")
df_consommation_numeric.filter(F.col("consommation_clean") > 15_000).groupBy("type_energie").count().show()

Valeurs negatives dans la colonne consommation:
+------------+-----+
|type_energie|count|
+------------+-----+
|         eau|12945|
|         gaz|12997|
| electricite|12968|
+------------+-----+

Valeurs abérantes > 15 000 dans la colonne consommation :
+------------+-----+
|type_energie|count|
+------------+-----+
|         eau|12810|
|         gaz|12764|
| electricite|12986|
+------------+-----+



- Identifier les batiments avec le plus de mesures

In [29]:
top_batiment = (
    df_consommation_numeric.filter(F.col("consommation_clean").isNotNull())
    .groupBy("batiment_id")
    .agg(F.count("*").alias("count"))
    .orderBy(F.col("count").desc())
)

print("Top 10 des bâtiments avec le plus de mesure :")
top_batiment.show(10)

Top 10 des bâtiments avec le plus de mesure :
+-----------+-----+
|batiment_id|count|
+-----------+-----+
|    BAT0086|53015|
|    BAT0002|53002|
|    BAT0093|52993|
|    BAT0051|52989|
|    BAT0117|52988|
|    BAT0146|52985|
|    BAT0052|52974|
|    BAT0145|52969|
|    BAT0047|52964|
|    BAT0061|52963|
+-----------+-----+
only showing top 10 rows



- Produire un rapport d'audit de qualite des donnees

In [ ]:
total = df_consommations_raw.count()

# Valeurs non numeriques
non_numeric = df_consommations_raw.filter(~F.col("consommation").rlike("^-?[0-9]+[.,]?[0-9]*$")).count()

# Valeurs avec virgule
with_comma = df_consommations_raw.filter(F.col("consommation").rlike("^-?[0-9]+[.,]?[0-9]*$")).filter(F.col("consommation").contains(",")).count()

# Valeurs negatives (apres conversion)
negative = df_consommation_numeric.filter(F.col("consommation_clean") < 0).count()

# Valeurs aberrantes > 15 000
outliers = df_consommation_numeric.filter(F.col("consommation_clean") > 15_000).count()

# Doublons
duplicates = total - df_consommation_numeric_timestamp.dropDuplicates(["batiment_id", "timestamp_clean", "type_energie"]).count()


print(f"Total enregistrements: {total:,}")
print()
print(f"Problemes identifies:")
print(f"  - Valeurs non numeriques: {non_numeric:,} ({non_numeric/total*100:.2f}%)")
print(f"  - Valeurs avec virgule decimale: {with_comma:,} ({with_comma/total*100:.2f}%)")
print(f"  - Valeurs negatives: {negative:,} ({negative/total*100:.2f}%)")
print(f"  - Valeurs aberrantes (> 15 000): {outliers:,} ({outliers/total*100:.2f}%)")
print(f"  - Doublons: {duplicates:,} ({duplicates/total*100:.2f}%)")
print(f"  - Formats de dates multiples: 4 formats differents detectes")

Total enregistrements: 7,758,868

Problemes identifies:
  - Valeurs non numeriques: 38,975 (0.50%)
  - Valeurs avec virgule decimale: 925,392 (11.93%)
  - Valeurs negatives: 38,910 (0.50%)
  - Valeurs aberrantes (>1000): 38,560 (0.50%)
  - Doublons: 152,995 (1.97%)
  - Formats de dates multiples: 4 formats differents detectes


In [42]:
spark.stop()